In [1]:
!pip install qiskit qiskit_aer
!pip install pylatexenc


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=a9bbf1458274e67af3707b4a5b00222f63c30f56e446ebd72c9776b3085b55f0
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [2]:
# Cell 2 — imports and helper functions (QFT and inverse QFT)
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

def qft_circuit(n):
    """
    Return a QuantumCircuit that performs QFT on n qubits (without final measurement).
    The circuit acts on qubits 0..n-1 (0 is MSB here for visualization).
    """
    qc = QuantumCircuit(n)
    # QFT implementation: apply H and controlled rotations
    for j in range(n):
        qc.h(j)
        # controlled rotations between j and k>j
        for k in range(1, n-j):
            angle = np.pi / (2**k)
            qc.cp(angle, j+k, j)  # controlled phase: control qubit (j+k) -> target (j)
            # note: cp(control, target) ordering in some qiskit versions is qc.cp(theta, control, target)
            # Here we use the ordering qc.cp(angle, control, target).
    # swap qubits to reverse order
    for i in range(n//2):
        qc.swap(i, n-1-i)
    return qc

def inverse_qft_circuit(n):
    """
    Return the inverse QFT circuit on n qubits.
    """
    qc = QuantumCircuit(n)
    # swaps
    for i in range(n//2):
        qc.swap(i, n-1-i)
    # reverse of QFT
    for j in reversed(range(n)):
        # controlled rotations reversed and negated
        for k in range(1, n-j):
            angle = -np.pi / (2**k)
            qc.cp(angle, j+k, j)
        qc.h(j)
    return qc


In [3]:
for n in [2, 3, 4]:
    qc = qft_circuit(n)
    print(qc.draw('text'))


     ┌───┐                 
q_0: ┤ H ├─■─────────────X─
     └───┘ │P(π/2) ┌───┐ │ 
q_1: ──────■───────┤ H ├─X─
                   └───┘   
     ┌───┐                                        
q_0: ┤ H ├─■────────■───────────────────────────X─
     └───┘ │P(π/2)  │       ┌───┐               │ 
q_1: ──────■────────┼───────┤ H ├─■─────────────┼─
                    │P(π/4) └───┘ │P(π/2) ┌───┐ │ 
q_2: ───────────────■─────────────■───────┤ H ├─X─
                                          └───┘   
     ┌───┐                                                                     »
q_0: ┤ H ├─■────────■─────────────■────────────────────────────────────────────»
     └───┘ │P(π/2)  │       ┌───┐ │                                            »
q_1: ──────■────────┼───────┤ H ├─┼────────■────────■───────────────────────X──»
                    │P(π/4) └───┘ │        │P(π/2)  │       ┌───┐           │  »
q_2: ───────────────■─────────────┼────────■────────┼───────┤ H ├─■─────────X──»
                 

In [9]:
# Cell 4 — Test QFT + inverse QFT (sanity check): apply QFT, then inverse QFT and measure that we get back the input state.
def test_qft_inverse(n, input_state_index=3):
    """
    Build circuit of n qubits with initial computational basis state |input_state_index>,
    apply qft, inverse qft, and measure. Returns counts.
    """
    qr = QuantumRegister(n, 'q')
    cr = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qr, cr)
    # prepare computational basis state
    bits = format(input_state_index, f'0{n}b')[::-1]  # reverse bit order for q[0] being LSB here
    for i, b in enumerate(bits):
        if b == '1':
            qc.x(i)
    # apply QFT then inverse
    qc.compose(qft_circuit(n), inplace=True)
    qc.compose(inverse_qft_circuit(n), inplace=True)
    qc.measure(qr, cr)
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=2048).result()
    counts = result.get_counts()
    return qc, counts

# Example: n=3, input state |3>
qc, counts = test_qft_inverse(3, input_state_index=3)
print("Counts after QFT followed by inverse QFT (should return the input state |3>):")
print(counts)
display(qc.draw('text'))
plot_histogram(counts)
plt.show()


Counts after QFT followed by inverse QFT (should return the input state |3>):
{'011': 2048}


┌───┐┌───┐                                                               »
q_0: ┤ X ├┤ H ├─■────────■───────────────────────────X──X─────────────────────»
     ├───┤└───┘ │P(π/2)  │       ┌───┐               │  │                ┌───┐»
q_1: ┤ X ├──────■────────┼───────┤ H ├─■─────────────┼──┼───────■────────┤ H ├»
     └───┘               │P(π/4) └───┘ │P(π/2) ┌───┐ │  │ ┌───┐ │P(-π/2) └───┘»
q_2: ────────────────────■─────────────■───────┤ H ├─X──X─┤ H ├─■─────────────»
                                               └───┘      └───┘               »
c: 3/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«                         ┌───┐   ┌─┐
«q_0: ─■─────────■────────┤ H ├───┤M├
«      │P(-π/2)  │        └┬─┬┘   └╥┘
«q_1: ─■─────────┼─────────┤M├─────╫─
«                │P(-π/4)  └╥┘ ┌─┐ ║ 
«q_2: ───────────■──────────╫──┤M├─╫─
«                           ║  └╥┘ ║ 
«c: 3/══════════════════════╩═══╩══╩═
«                           1   2  0

In [7]:
# Cell 5 — Apply QFT and measure the result distribution for different input basis states
def qft_measure_counts(n, input_state_index=1, shots=2048):
    qr = QuantumRegister(n, 'q')
    cr = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qr, cr)
    # prepare input state
    bits = format(input_state_index, f'0{n}b')[::-1]
    for i, b in enumerate(bits):
        if b == '1':
            qc.x(i)
    qc.append(qft_circuit(n).to_gate(), qr)  # apply QFT
    qc.measure(qr, cr)
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    return qc, result.get_counts()

# show probability distribution for QFT applied to |1> for n=3
qc, counts = qft_measure_counts(3, input_state_index=1)
print("QFT applied to |1> on 3 qubits -> measured distribution:")
print(counts)
display(qc.draw('text'))
plot_histogram(counts)
plt.show()


QFT applied to |1> on 3 qubits -> measured distribution:
{'110': 241, '101': 234, '001': 272, '000': 243, '010': 266, '011': 262, '111': 277, '100': 253}


┌───┐┌─────────────┐┌─┐      
q_0: ┤ X ├┤0            ├┤M├──────
     └───┘│             │└╥┘┌─┐   
q_1: ─────┤1 circuit-64 ├─╫─┤M├───
          │             │ ║ └╥┘┌─┐
q_2: ─────┤2            ├─╫──╫─┤M├
          └─────────────┘ ║  ║ └╥┘
c: 3/═════════════════════╩══╩══╩═
                          0  1  2

In [10]:
# Cell 6 — Phase Estimation using QFT (integration with phase estimation)
# We use a simple unitary U = diag(1, e^{2π i θ}) with eigenstate |1> having phase θ.
# The phase estimation circuit estimates θ. We use t counting qubits and 1 target qubit.

def controlled_phase_power(qc, control, target, theta, power):
    """
    Implements controlled-U^{2^power} where U = diag(1, e^{2π i theta}).
    Controlled phase gate with angle = 2π * theta * 2^power applied if target is |1>.
    """
    angle = 2 * np.pi * theta * (2**power)
    qc.cp(angle, control, target)

def phase_estimation_circuit(theta, t=3):
    """
    Build a phase estimation circuit that uses t counting qubits and 1 target qubit.
    The unitary has eigenvalue e^{2π i theta} on |1>.
    Returns qc and number of counting qubits t.
    """
    # registers
    counting = QuantumRegister(t, 'count')
    target = QuantumRegister(1, 'target')
    cr = ClassicalRegister(t, 'c')
    qc = QuantumCircuit(counting, target, cr)
    # prepare target in eigenstate |1> (the eigenstate of U with phase theta)
    qc.x(target[0])
    # put counting qubits into |+>
    qc.h(counting)
    # apply controlled-U^(2^j)
    for j in range(t):
        # apply controlled-U^{2^{t-1-j}} to counting[j] (most significant first)
        power = t-1-j
        controlled_phase_power(qc, counting[j], target[0], theta, power)
    # inverse QFT on counting register
    inv_qft = inverse_qft_circuit(t)
    qc.append(inv_qft.to_gate(), counting)
    qc.measure(counting, cr)
    return qc

# Example: theta = 0.125 (i.e., exact binary .001 for t>=3)
theta = 0.125
t = 3  # number of counting qubits (precision)
qc_pe = phase_estimation_circuit(theta, t=t)
display(qc_pe.draw('text'))
sim = AerSimulator()
tqc = transpile(qc_pe, sim)
result = sim.run(tqc, shots=4096).result()
counts_pe = result.get_counts()
print("Phase estimation counts (binary encoded estimate of theta):")
print(counts_pe)
plot_histogram(counts_pe)
plt.show()

# Convert most probable binary string to phase estimate
most_prob_str = max(counts_pe, key=counts_pe.get)
# Note: Qiskit returns strings with leftmost being latest qubit ordering; ensure we interpret correctly.
estimated_decimal = int(most_prob_str, 2) / (2**t)
print(f"Most probable measured bitstring: {most_prob_str} -> estimated phase = {estimated_decimal:.6f}")
print(f"True theta = {theta}")


┌───┐                         ┌─────────────┐┌─┐      
count_0: ┤ H ├─■───────────────────────┤0            ├┤M├──────
         ├───┤ │                       │             │└╥┘┌─┐   
count_1: ┤ H ├─┼──────■────────────────┤1 circuit-78 ├─╫─┤M├───
         ├───┤ │      │                │             │ ║ └╥┘┌─┐
count_2: ┤ H ├─┼──────┼────────■───────┤2            ├─╫──╫─┤M├
         ├───┤ │P(π)  │P(π/2)  │P(π/4) └─────────────┘ ║  ║ └╥┘
 target: ┤ X ├─■──────■────────■───────────────────────╫──╫──╫─
         └───┘                                         ║  ║  ║ 
    c: 3/══════════════════════════════════════════════╩══╩══╩═
                                                       0  1  2

Phase estimation counts (binary encoded estimate of theta):
{'100': 4096}
Most probable measured bitstring: 100 -> estimated phase = 0.500000
True theta = 0.125
